In [ ]:
import pandas as pd
import statsapi
import datetime
import string
import json
import os

from bs4 import BeautifulSoup
import requests


In [ ]:
player_search = pd.read_csv("data\\player_search.csv", encoding = "ISO-8859-1")
team_search = pd.read_csv("data\\team_search.csv")

wOBA_constants = pd.read_csv("data\\wOBA_constants.csv", index_col=0)
park_factors = pd.read_csv("data\\park_factors.csv", index_col=0)
league_averages = pd.read_csv("data\\league_averages.csv", index_col=0)

with open('data\\batter_hands.json') as f:
    batter_hands = json.load(f)
with open('data\\pitcher_hands.json') as f:
    pitcher_hands = json.load(f)


In [ ]:
def adv_matchup(row):
    val = 0
    if row['batter_hand'] == 'S':
        val = 1
    elif row['batter_hand'] == 'R' and row['pitcher_hand'] == 'L':
        val = 1
    elif row['batter_hand'] == 'L' and row['pitcher_hand'] == 'R':
        val = 1
    return val

for file in os.scandir("data\\matchups"):
    matchups = pd.read_csv(file, index_col=0)
    matchups['adv_ab'] = matchups.apply(adv_matchup, axis=1)
    matchups.to_csv(f"data\\matchups\\{file.name}")
